# **앙상블(Ensemble) 학습**

- 정형 데이터를 다루는 데 가장 뛰어난 알고리즘

<br>

## **1. 랜덤 포레스트(Random Forest)**

- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다
- 각 결정 트리의 예측을 사용해 최종 예측을 만든다
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 골라 최선의 분할을 찾는다
- RandomForestClassifier 클래스
  - 먼저 각 트리를 훈련하기 위한 데이터를 랜덤하게 만든다
  - **부트스트랩 샘플(bootstrap sample)** : 입력한 훈련 데이터에서 샘플을 추출하여 훈련 데이터를 만드는 것. 중복되어 추출될 수 있음
  - 예. 1,000개의 샘플이 있는 가방에서 100개를 뽑는다면, 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣는다
  - 부트스트랩 샘플은 훈련 세트와 크기가 같게 만든다


In [2]:
# 화이트 와인 분류 데이터에 랜덤 포레스트 적용
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
# return_train_score=True는 검증 점수와 훈련 세트 점수도 같이 반환(기본값은 False)
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
# 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier의 중요 매개변수가 모두 존재함
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


### **1.1 OOB 샘플(Out of Bag sample)**

- 랜덤 포레스트 사용 시 부트스트랩 샘플에 포함되지 않은 남은 샘플
- 이 샘플로 훈련된 모델을 평가할 수 있음 = 검증 세트
- 이 점수를 얻으려면 oob_score=True 지정


In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
# oob_score를 출력하면 교차 검증을 대신할 수 있다
print(rf.oob_score_)

0.8934000384837406


## **2. 엑스트라 트리(Extra Tree)**

- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용
  - 기본적으로 100개의 트리 훈련
- 랜덤 포레스트와 비슷하지만 부트스트랩 샘플을 사용하지 않음
- 노드를 분할할 때 무작위로 분할함
  - 그러면 성능이 떨어지지 않나? -> 많은 수의 트리를 앙상블하기 때문에 오히려 과적합을 방지하고 검증 세트의 점수를 높임
  - 무작위성 덕분에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야하지만 계산은 더 빠름
- DecisionTreeClassifier에서 splitter 인자를 random으로 주었다면 이것이 엑스트라 트리
- ExtraTreesClassifier 클래스
  - 회귀 버전은 ExtraTreesRegressor


In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
# 특성 중요도 확인
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## **3. 그레이디언트 부스팅(Gradient Boosting)**

- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
- 기본적으로 깊이 3, 결정 트리 100개 이용
- 과대적합에 강하고 일반화 성능이 높음
- 경사 하강법을 사용하여 트리를 앙상블에 추가
  - 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동 -> 그래서 깊이가 얕은 트리를 사용
  - 성능은 뛰어나지만 속도는 앞선 두 알고리즘보다 느림
- 분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수 사용
- GradientBoostingClassifier 클래스, 회귀 버전은 GradientBoostingRegressor
  - subsample은 사용할 훈련 세트의 비율을 정함 -> 1보다 작으면 훈련 세트의 일부만 사용
  - 학습률 매개변수로 속도 조절
  - n_jobs는 없음 -> 순서대로 트리를 추가하기 때문에 속도가 느림


In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# 과대적합에 강한 모습
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
# 학습률(기본값=0.1)을 높이고 트리 개수를 증가시키면 성능이 향상될 수 있음
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [11]:
# 특성 중요도 확인
# 랜덤 포레스트나 엑스트라 트리와는 달리 일부 특성에 더 의존함
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## **4. 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)**

- 그레이디언트 부스팅의 속도와 성능을 개선한 버전
- 먼저 입력 특성을 256개의 구간으로 나눔
- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함 -> 누락된 특성이 있더라도 전처리 필요x
- HistGradientBoostingClassifier 클래스, 회귀 버전은 HistGradientBoostingRegressor
- 성능 향상을 위한 매개변수
  - max_iter : 부스팅 반복 횟수 지정(트리의 개수를 지정)

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
# 과대적합을 억압하면서 그레이디언트 부스팅보다는 조금 나은 성능을 보임
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [16]:
# 특성 중요도 확인을 위해 permutation_importance() 함수 사용
# permutation_importance() : 특성을 하나씩 랜덤하게 섞어 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산
# 테스트 세트에도 적용 가능
# n_repeats 매개변수 : 랜덤하게 섞을 횟수를 지정
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
# permutation_importance() 반환 객체 : 특성 중요도(importances), 평균(importances_mean), 표준편차(importances_std)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [18]:
# 테스트 세트의 특성 중요도 확인
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
# 최종 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

### **4.1 XGBoost**

- 사이킷런이 아닌 그레이디언트 부스팅 알고리즘을 구현한 라이브러리
- cross_validate함수와 함께 사용 가능
- tree_method 매개변수를 hist로 지정하면 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있음

### **4.2 LightGBM**

- 마이크로소프트에서 만든 히스토그램 기반 그레이디언트 부스팅 라이브러리
- 사이킷런의 라이브러리가 영향을 많이 받음

In [19]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [22]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
